In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import detector
import matplotlib.pyplot as plt
import cv2
import skvideo.io as io

In [ ]:
num1 = 76
num2 = 79

im1 = 'imgs/{}_cam-image_array_.jpg'.format(num1)
im2 = 'imgs/{}_cam-image_array_.jpg'.format(num2)
depth1 = 'imgs/{}_cam-depth_array_.png'.format(num1)
depth2 = 'imgs/{}_cam-depth_array_.png'.format(num2)

im1 = cv2.imread(im1, cv2.IMREAD_GRAYSCALE)
im2 = cv2.imread(im2, cv2.IMREAD_GRAYSCALE)
depth1 = cv2.imread(depth1, -1)
depth2 = cv2.imread(depth2, -1)


In [ ]:
detect = detector.Detector(im1, im2, depth1, depth2, detector='fast')
# detect = detector.Detector(im1, im2)
kp1, des1, kp2, des2 = detect.feature_extraction()

In [ ]:
matches = detect.fMatcher.knnMatch(des2, des1, k=2)
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
# matches = sorted(matches, key=lambda x: x.distance)[:400]
img3 = cv2.drawMatches(detect.img1,kp1,detect.img2,kp2,good[:300],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(18, 12))
plt.imshow(img3)

In [ ]:
num = len(good)
pts1 = np.zeros((num, 2), dtype=np.float32)
pts2 = np.zeros((num, 2), dtype=np.float32)

for i, match in enumerate(good):
    pts1[i, :] = kp1[match.trainIdx].pt
    pts2[i, :] = kp2[match.queryIdx].pt
    
H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC)
size = im1.shape
warped_im1 = cv2.warpPerspective(im1, H, (size[1], size[0]))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 12))
ax[0].imshow(im1, cmap='gray')
ax[1].imshow(warped_im1, cmap='gray')
ax[2].imshow(im2, cmap='gray')

In [ ]:
warped_im1_blur = cv2.GaussianBlur(warped_im1, (5, 5), 0)
im2_blur = cv2.GaussianBlur(im2, (5, 5), 0)

normalized_im1 = (warped_im1_blur - np.mean(warped_im1_blur)) / np.std(warped_im1_blur) 
normalized_im2 = (im2_blur - np.mean(im2_blur)) / np.std(im2_blur)
diff = (np.abs(normalized_im1 - normalized_im2) * 255).astype(np.uint8)
if np.max(diff) - np.min(diff) >= 250:
    result = cv2.threshold(diff,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
else:
    result = np.zeros(im2_blur.shape)
plt.imshow(result, cmap='gray')


In [ ]:
def detect(im1, im2):
#     detect = detector.Detector(im1, im2, depth1, depth2, detector='fast')
    detect = detector.Detector(im1, im2)
    kp1, des1, kp2, des2 = detect.feature_extraction()
    
    matches = detect.fMatcher.knnMatch(des2, des1, k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)
            
    num = len(good)
    pts1 = np.zeros((num, 2), dtype=np.float32)
    pts2 = np.zeros((num, 2), dtype=np.float32)

    for i, match in enumerate(good):
        pts1[i, :] = kp1[match.trainIdx].pt
        pts2[i, :] = kp2[match.queryIdx].pt

    H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC)
    size = im1.shape
    warped_im1 = cv2.warpPerspective(im1, H, (size[1], size[0]))
    
    warped_im1_blur = cv2.GaussianBlur(warped_im1, (5, 5), 0)
    im2_blur = cv2.GaussianBlur(im2, (5, 5), 0)

    normalized_im1 = (warped_im1_blur - np.mean(warped_im1_blur)) / np.std(warped_im1_blur) 
    normalized_im2 = (im2_blur - np.mean(im2_blur)) / np.std(im2_blur)
    diff = (np.abs(normalized_im1 - normalized_im2) * 255).astype(np.uint8)
    if np.max(diff) - np.min(diff) >= 250:
        result = cv2.threshold(diff,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    else:
        result = np.zeros(im2_blur.shape)
        
    return result

In [ ]:
# imgs = []
# for i in range(70, 340):
#     filename = 'imgs/{}_cam-image_array_.jpg'.format(i)
#     img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
#     imgs.append(img)
# io.vwrite("walking.MOV", imgs) 

images = io.vread('videos/chicken_ds1.MOV', as_grey=True)
print(images.shape)
    
video = []
failure = 0
for i, frame in enumerate(images):
    frame = frame[:, :, 0]
    if i < 3:
        detected = np.zeros(frame.shape)
    else:
        try:
            detected = detect(images[i-3], frame)
        except:
            failure += 1
            detected = np.zeros(frame.shape)

    img = np.vstack((frame, detected))
    video.append(img)

print("failure: ", failure)

In [ ]:
io.vwrite('videos/chicken_ds1_detected.MOV', video)
# io.vwrite('videos/walking_detected_short.MOV', video[:71])
# io.vwrite('videos/walking_detected_median.MOV', video[:130])

In [ ]:
plt.figure(figsize=(10, 20))
plt.imshow(video[6], cmap='gray')